# Prepare

In [1]:
!nvidia-smi

Mon Jan  2 14:21:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%cd "/content/drive/MyDrive/CE Project"
!ls

/content/drive/MyDrive/CE Project
 data		 softmax_temperature.ipynb   svm3.ipynb  'Use Library.ipynb'
 softmax.ipynb	 svm2.ipynb		     SVM.ipynb


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import time
from sklearn.metrics import accuracy_score, f1_score
from keras.datasets import mnist, cifar10
from sklearn.model_selection import train_test_split
import torch
from sklearn.preprocessing import OneHotEncoder
import random

In [4]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed_everything(22)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define model

In [71]:
class SoftmaxTemperature:
    def softmax(self, Z):
        e_z = torch.exp(Z/self.T)
        A = e_z / e_z.sum(dim=1, keepdim=True)
        return A

    def loss(self):
        A = self.softmax(torch.mm(self.X, self.W))
        return -torch.mean(torch.sum(self.y * torch.log(A), dim=1))

    def grad(self, X, y):
        A = self.softmax(torch.mm(X, self.W))
        A = A - y
        return torch.mm(X.T, A.to(torch.float32)) / X.shape[0]

    def fit(self, X, y, W, T, lr=0.05, epochs=200, tol=1e-5, batch_size=32):
        self.X = X
        self.y = y
        self.W = torch.clone(W)
        self.T = T

        ep = 0
        # loss_hist = [self.loss().cpu().numpy()]
        N = X.shape[0]
        batches = int(np.ceil(N/batch_size))
        
        while ep < epochs:
            ep += 1
            mix_ids = torch.randperm(N)

            for i in range(batches):
                batch_ids = mix_ids[batch_size*i : min(batch_size*(i+1), N)]
                X_batch, y_batch = X[batch_ids], y[batch_ids]
                self.W -= lr * self.grad(X_batch, y_batch)

            # loss_hist.append(self.loss().cpu().numpy())
            if ep % 10 == 0:
                print('Epoch:', ep, 'loss:', self.loss().cpu().numpy())

    def parameters(self):
        return self.W
    
    def predict(self, X):
        A = self.softmax(torch.mm(X, self.W))
        return torch.argmax(A, dim=1)

# 1. MNIST

In [11]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(y_train[0])
le = OneHotEncoder()
y_train = le.fit_transform(y_train.reshape(-1, 1)).toarray()
print(y_train[0])

X_train = X_train.reshape(-1, 784)
X_train = X_train.astype('float32')
X_test = X_test.reshape(-1, 784)
X_test = X_test.astype('float32')

# normalize
X_train /= 255
X_test /= 255
print('Train samples:', X_train.shape)
print('Test samples:', X_test.shape)

5
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
Train samples: (60000, 784)
Test samples: (10000, 784)


In [12]:
X_train = torch.from_numpy(X_train).to(device)
y_train = torch.from_numpy(y_train).to(device)
X_test = torch.from_numpy(X_test).to(device)
# y_test = torch.from_numpy(y_test).to(device)

In [45]:
W_init = torch.randn(X_train.shape[1], 10).to(device)

### 1.5

In [65]:
lr = 0.05
T = 1.5
start = time.time()
model = SoftmaxTemperature()
model.fit(X_train, y_train, W_init, T)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 10 loss: 0.37848175493278613
Epoch: 20 loss: 0.32794696158467945
Epoch: 30 loss: 0.30456451219732106
Epoch: 40 loss: 0.29169036609171006
Epoch: 50 loss: 0.28226624238043546
Epoch: 60 loss: 0.27579345812201744
Epoch: 70 loss: 0.2697551260602253
Epoch: 80 loss: 0.2672357201224741
Epoch: 90 loss: 0.2626967353229213
Epoch: 100 loss: 0.2591680316199559
Epoch: 110 loss: 0.2567020108772778
Epoch: 120 loss: 0.25542291837891634
Epoch: 130 loss: 0.25286485592049396
Epoch: 140 loss: 0.2523706238914793
Epoch: 150 loss: 0.2505077417606611
Epoch: 160 loss: 0.24922722138139847
Epoch: 170 loss: 0.24756123758710227
Epoch: 180 loss: 0.24615438845509352
Epoch: 190 loss: 0.24650107444036432
Epoch: 200 loss: 0.24581068399239722
Train completed in 83.35889482498169s


In [66]:
y_pred = model.predict(X_test)
accuracy_score(y_pred.cpu().numpy(), y_test)*100

92.33

### 1000

In [48]:
T = 1000
start = time.time()
model = SoftmaxTemperature()
model.fit(X_train, y_train, W_init, T)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 5 loss: 1.8807726835628351
Epoch: 10 loss: 1.5867530303309363
Epoch: 15 loss: 1.3785649268552662
Epoch: 20 loss: 1.2280219563198587
Epoch: 25 loss: 1.1158879554931076
Epoch: 30 loss: 1.0297873952642083
Epoch: 35 loss: 0.9618237614115079
Epoch: 40 loss: 0.9068740078600744
Epoch: 45 loss: 0.8615230748217554
Epoch: 50 loss: 0.8234358729486665
Epoch: 55 loss: 0.7909663897349034
Epoch: 60 loss: 0.762929677682432
Epoch: 65 loss: 0.7384506008479279
Epoch: 70 loss: 0.7168709259980902
Epoch: 75 loss: 0.6976856414338497
Epoch: 80 loss: 0.6805016684888552
Epoch: 85 loss: 0.6650083546788471
Epoch: 90 loss: 0.6509565997947085
Epoch: 95 loss: 0.638144821268553
Epoch: 100 loss: 0.6264077968010873
Epoch: 105 loss: 0.6156086838240018
Epoch: 110 loss: 0.6056333508502265
Epoch: 115 loss: 0.5963859591127141
Epoch: 120 loss: 0.5877849580183082
Epoch: 125 loss: 0.5797609451374001
Epoch: 130 loss: 0.5722542080176898
Epoch: 135 loss: 0.5652131101440319
Epoch: 140 loss: 0.5585929582950688
Epoch: 145 los

In [49]:
y_pred = model.predict(X_test)
accuracy_score(y_pred.cpu().numpy(), y_test)*100

88.29

In [69]:
torch.max(W_init)

tensor(4.2385, device='cuda:0')

In [70]:
print(torch.max(model.parameters()))

tensor(3.9304, device='cuda:0')


### 0.9

In [55]:
T = 0.9
start = time.time()
model = SoftmaxTemperature()
model.fit(X_train, y_train, W_init, T)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 10 loss: 0.47695634076249094
Epoch: 20 loss: 0.3843804617118477
Epoch: 30 loss: 0.3454000725188925
Epoch: 40 loss: 0.3195931773959169
Epoch: 50 loss: 0.3015944909856845
Epoch: 60 loss: 0.2900990746686076
Epoch: 70 loss: 0.28044448619644924
Epoch: 80 loss: 0.27514573440705614
Epoch: 90 loss: 0.2722838023490485
Epoch: 100 loss: 0.26429458432205566
Epoch: 110 loss: 0.26025947427596796
Epoch: 120 loss: 0.25865700085919147
Epoch: 130 loss: 0.2569744502157121
Epoch: 140 loss: 0.25457369244847167
Epoch: 150 loss: 0.25355132451616696
Epoch: 160 loss: 0.2488930980302461
Epoch: 170 loss: 0.248742405865024
Epoch: 180 loss: 0.2466658634264443
Epoch: 190 loss: 0.24454946060467547
Epoch: 200 loss: 0.24410225776852693
Train completed in 83.27100038528442s


In [56]:
y_pred = model.predict(X_test)
accuracy_score(y_pred.cpu().numpy(), y_test)*100

92.4

### 0.8

In [67]:
T = 0.8
start = time.time()
model = SoftmaxTemperature()
model.fit(X_train, y_train, W_init, T)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 10 loss: 0.5125927390215458
Epoch: 20 loss: 0.40505886529347634
Epoch: 30 loss: 0.35713882765092975
Epoch: 40 loss: 0.3268677756345718
Epoch: 50 loss: 0.30865782176638995
Epoch: 60 loss: 0.2963359972715008
Epoch: 70 loss: 0.2850425697341067
Epoch: 80 loss: 0.27800346286740885
Epoch: 90 loss: 0.2714646445745294
Epoch: 100 loss: 0.2679925311627106
Epoch: 110 loss: 0.26310337800482875
Epoch: 120 loss: 0.26034272905324957
Epoch: 130 loss: 0.25653494492023593
Epoch: 140 loss: 0.2548942416237584
Epoch: 150 loss: 0.25234121154727
Epoch: 160 loss: 0.2522611370649494
Epoch: 170 loss: 0.24961674055106742
Epoch: 180 loss: 0.24774518898136386
Epoch: 190 loss: 0.2470468377926977
Epoch: 200 loss: 0.24590904019025142
Train completed in 84.55078935623169s


In [68]:
y_pred = model.predict(X_test)
accuracy_score(y_pred.cpu().numpy(), y_test)*100

92.2

# Flower

In [72]:
# Đường dẫn
features_path = '/content/drive/MyDrive/CE Project/data/flower/flower_features.sav'
labels_path = '/content/drive/MyDrive/CE Project/data/flower/flower_labels.sav'

# Load dữ liệu
features = np.array(pickle.load(open(features_path, 'rb')))
labels = np.array(pickle.load(open(labels_path, 'rb')))

In [145]:
# Chia traing set, test set tỉ lệ 80-20
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [146]:
max_values = np.max(X_train)
min_values = np.min(X_train)
a, b = 0, 1
X_train = a + (X_train - min_values) * (b-a) / (max_values - min_values)
X_test = a + (X_test - min_values) * (b-a) / (max_values - min_values)

In [147]:
print(y_train[0])
le = OneHotEncoder()
y_train = le.fit_transform(y_train.reshape(-1, 1)).toarray()
print(y_train[0])
print('Train samples:', X_train.shape)
print('Test samples:', X_test.shape)

2
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Train samples: (1088, 25088)
Test samples: (272, 25088)


In [148]:
X_train = torch.from_numpy(X_train).to(device)
y_train = torch.from_numpy(y_train).to(device)
X_test = torch.from_numpy(X_test).to(device)
# y_test = torch.from_numpy(y_test).to(device)

In [149]:
W_init = torch.randn(X_train.shape[1], y_train.shape[1]).to(device)

### 1.5

In [151]:
T = 1.5
start = time.time()
model = SoftmaxTemperature()
model.fit(X_train, y_train, W_init, T, lr=60)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 10 loss: 0.0031689919006495453
Epoch: 20 loss: 0.0018581670156590218
Epoch: 30 loss: 0.0013439331716298016
Epoch: 40 loss: 0.0010614095740303952
Epoch: 50 loss: 0.0008807176845366673
Epoch: 60 loss: 0.0007545195868823296
Epoch: 70 loss: 0.0006610518474160433
Epoch: 80 loss: 0.0005888798896975016
Epoch: 90 loss: 0.0005313856907447998
Epoch: 100 loss: 0.0004844459905412895
Epoch: 110 loss: 0.0004453666276812625
Epoch: 120 loss: 0.0004122992068713321
Epoch: 130 loss: 0.00038393924279895723
Epoch: 140 loss: 0.0003593379241246416
Epoch: 150 loss: 0.00033778233474411693
Epoch: 160 loss: 0.00031873621044996384
Epoch: 170 loss: 0.00030177758697257816
Epoch: 180 loss: 0.0002865800605331703
Epoch: 190 loss: 0.000272882964220978
Epoch: 200 loss: 0.000260465086747786
Train completed in 2.4282004833221436s


In [152]:
y_pred = model.predict(X_test)
accuracy_score(y_pred.cpu().numpy(), y_test)*100

91.54411764705883

### 1000

In [153]:
T = 1000
start = time.time()
model = SoftmaxTemperature()
model.fit(X_train, y_train, W_init, T, lr=60)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 10 loss: 1.848231014686034
Epoch: 20 loss: 1.386983051752288
Epoch: 30 loss: 1.1162471444815096
Epoch: 40 loss: 0.9376552681291394
Epoch: 50 loss: 0.8107965761288035
Epoch: 60 loss: 0.7157964854112138
Epoch: 70 loss: 0.6417420965177264
Epoch: 80 loss: 0.5821904727089371
Epoch: 90 loss: 0.5331032271417236
Epoch: 100 loss: 0.49183462313425585
Epoch: 110 loss: 0.45657541281118885
Epoch: 120 loss: 0.42604676961379
Epoch: 130 loss: 0.3993163850384864
Epoch: 140 loss: 0.37569077302741255
Epoch: 150 loss: 0.3546397445123496
Epoch: 160 loss: 0.3357500857984552
Epoch: 170 loss: 0.3186970291010044
Epoch: 180 loss: 0.30321795470418317
Epoch: 190 loss: 0.28910110055456995
Epoch: 200 loss: 0.2761710252442794
Train completed in 2.339296817779541s


In [154]:
y_pred = model.predict(X_test)
accuracy_score(y_pred.cpu().numpy(), y_test)*100

88.60294117647058

### 0.9

In [161]:
T = 0.9
start = time.time()
model = SoftmaxTemperature()
model.fit(X_train, y_train, W_init, T, lr=30)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 10 loss: 0.004179743478599572
Epoch: 20 loss: 0.0023764192610149113
Epoch: 30 loss: 0.0016969712726855846
Epoch: 40 loss: 0.0013299079594955936
Epoch: 50 loss: 0.0010980078827915218
Epoch: 60 loss: 0.0009372419966495476
Epoch: 70 loss: 0.0008188811160193739
Epoch: 80 loss: 0.0007279126894963461
Epoch: 90 loss: 0.0006556710441096063
Epoch: 100 loss: 0.000596865046696015
Epoch: 110 loss: 0.0005480186634524366
Epoch: 120 loss: 0.0005067656252265247
Epoch: 130 loss: 0.00047145128869700604
Epoch: 140 loss: 0.00044085649158875917
Epoch: 150 loss: 0.00041408660201933936
Epoch: 160 loss: 0.0003904602953163932
Epoch: 170 loss: 0.00036944844116130126
Epoch: 180 loss: 0.0003506341062520366
Epoch: 190 loss: 0.0003336863332801385
Epoch: 200 loss: 0.00031834279102130813
Train completed in 2.0969278812408447s


In [163]:
y_pred = model.predict(X_test)
accuracy_score(y_pred.cpu().numpy(), y_test)*100

86.76470588235294

# vnnews

In [164]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import LabelEncoder

In [165]:
# Đường dẫn
train_x_path = '/content/drive/MyDrive/CE Project/data/vnnews/train_x.sav'
train_y_path = '/content/drive/MyDrive/CE Project/data/vnnews/train_y.sav'
test_x_path = '/content/drive/MyDrive/CE Project/data/vnnews/test_x.sav'
test_y_path = '/content/drive/MyDrive/CE Project/data/vnnews/test_y.sav'

# Load dữ liệu
X_train = np.array(pickle.load(open(train_x_path, 'rb')))
y_train = np.array(pickle.load(open(train_y_path, 'rb')))
X_test = np.array(pickle.load(open(test_x_path, 'rb')))
y_test = np.array(pickle.load(open(test_y_path, 'rb')))

In [166]:
feature_extractor = Pipeline([
    # Chuyển tập dữ liệu sang dạng vector count dựa trên một vocab chung (BoW), loại bỏ những word có tần suất xuất hiện > 0.8
    ('vectorize', CountVectorizer(max_df=0.8, encoding='utf-16')),
    
    # Áp dụng TF-IDF để trích chọn đặc trưng (extract feature)
    ('feature extracter', TfidfTransformer())
])

X_train = feature_extractor.fit_transform(X_train, y_train)
X_test = feature_extractor.transform(X_test)
print('Kích thước vector input: ', X_train.shape)
print('Kích thước vector test', X_test.shape)

Kích thước vector input:  (14375, 79170)
Kích thước vector test (12076, 79170)


In [167]:
X_train = X_train.astype('float32').toarray()
X_test = X_test.astype('float32').toarray()

In [168]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [169]:
print(y_train[0])
le = OneHotEncoder()
y_train = le.fit_transform(y_train.reshape(-1, 1)).toarray()
print(y_train[0])
print('Train samples:', X_train.shape)
print('Test samples:', X_test.shape)

0
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
Train samples: (14375, 79170)
Test samples: (12076, 79170)


In [170]:
X_train = torch.from_numpy(X_train).to(device)
y_train = torch.from_numpy(y_train).to(device)
X_test = torch.from_numpy(X_test).to(device)
# y_test = torch.from_numpy(y_test).to(device)

In [171]:
W_init = torch.randn(X_train.shape[1], y_train.shape[1]).to(device)

### 1.5

In [172]:
T = 1.5
start = time.time()
model = SoftmaxTemperature()
model.fit(X_train, y_train, W_init, T, lr=50)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 10 loss: 0.028805466283359716
Epoch: 20 loss: 0.01628143187669392
Epoch: 30 loss: 0.012237578367236468
Epoch: 40 loss: 0.010150328142902822
Epoch: 50 loss: 0.009033612110320756
Epoch: 60 loss: 0.008219986058166323
Epoch: 70 loss: 0.007551642350303323
Epoch: 80 loss: 0.007155094860077607
Epoch: 90 loss: 0.006871222562153691
Epoch: 100 loss: 0.006605253841095933
Epoch: 110 loss: 0.006359006763075456
Epoch: 120 loss: 0.006185938501898503
Epoch: 130 loss: 0.006051948568316849
Epoch: 140 loss: 0.005842384488655979
Epoch: 150 loss: 0.006485920623421128
Epoch: 160 loss: 0.005625792889486732
Epoch: 170 loss: 0.0055847422269942925
Epoch: 180 loss: 0.0054009646989820114
Epoch: 190 loss: 0.005421983531571672
Epoch: 200 loss: 0.005383539058683488
Train completed in 59.21470594406128s


In [173]:
y_pred = model.predict(X_test)
accuracy_score(y_pred.cpu().numpy(), y_test)*100

90.48526001987412

### 1000

In [174]:
T = 1000
start = time.time()
model = SoftmaxTemperature()
model.fit(X_train, y_train, W_init, T, lr=50)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 10 loss: 2.5680764098561326
Epoch: 20 loss: 2.080091721141079
Epoch: 30 loss: 1.761962047043961
Epoch: 40 loss: 1.540485461144635
Epoch: 50 loss: 1.376177008833396
Epoch: 60 loss: 1.2484875123066097
Epoch: 70 loss: 1.1458945141675598
Epoch: 80 loss: 1.0613833223655575
Epoch: 90 loss: 0.9903979790009314
Epoch: 100 loss: 0.9297866459964733
Epoch: 110 loss: 0.8773566051068816
Epoch: 120 loss: 0.8314695553461899
Epoch: 130 loss: 0.7909254885816257
Epoch: 140 loss: 0.7547949787330438
Epoch: 150 loss: 0.7223725559132851
Epoch: 160 loss: 0.6930863696598032
Epoch: 170 loss: 0.6664803551139324
Epoch: 180 loss: 0.6421925909851824
Epoch: 190 loss: 0.6199191107283538
Epoch: 200 loss: 0.5994171886284638
Train completed in 61.74855875968933s


In [175]:
y_pred = model.predict(X_test)
accuracy_score(y_pred.cpu().numpy(), y_test)*100

88.47300430606161

### 0.9

In [176]:
T = 0.9
start = time.time()
model = SoftmaxTemperature()
model.fit(X_train, y_train, W_init, T, lr=50)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 10 loss: 0.01903146386214121
Epoch: 20 loss: 0.012016656612326775
Epoch: 30 loss: 0.009302844102647437
Epoch: 40 loss: 0.00890315499239418
Epoch: 50 loss: 0.00795965792615871
Epoch: 60 loss: 0.00731432185633487
Epoch: 70 loss: 0.007276156534744326
Epoch: 80 loss: 0.006506845108384886
Epoch: 90 loss: 0.006982589752183582
Epoch: 100 loss: 0.006251203388411201
Epoch: 110 loss: 0.0059100878963638075
Epoch: 120 loss: 0.006166849954270974
Epoch: 130 loss: 0.005935000075600146
Epoch: 140 loss: 0.00621279820477332
Epoch: 150 loss: 0.006051726046964766
Epoch: 160 loss: 0.005680470731590734
Epoch: 170 loss: 0.005753221361986697
Epoch: 180 loss: 0.0056118326376269735
Epoch: 190 loss: 0.005763767315442271
Epoch: 200 loss: 0.005598227494854409
Train completed in 59.3620662689209s


In [177]:
y_pred = model.predict(X_test)
accuracy_score(y_pred.cpu().numpy(), y_test)*100

89.83935077840344

# cifar10

In [178]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape, X_test.shape)

170498071/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3) (10000, 32, 32, 3)


In [179]:
X_train = np.array(pickle.load(open('/content/drive/MyDrive/CE Project/data/cifar10/train_features.sav', 'rb')))
y_train = np.array(pickle.load(open('/content/drive/MyDrive/CE Project/data/cifar10/y_train.sav', 'rb')))
X_test = np.array(pickle.load(open('/content/drive/MyDrive/CE Project/data/cifar10/test_features.sav', 'rb')))

print(X_train.shape, y_train.shape, X_test.shape)

(10000, 25088) (10000,) (10000, 25088)


In [180]:
maxX, minX = np.max(X_train), np.min(X_train)
a, b = 0, 1
X_train = a + (X_train - minX) * (b-a) / (maxX - minX)
X_test = a + (X_test - minX) * (b-a) / (maxX - minX)

In [181]:
print(y_train[0])
le = OneHotEncoder()
y_train = le.fit_transform(y_train.reshape(-1, 1)).toarray()
print(y_train[0])
print('Train samples:', X_train.shape)
print('Test samples:', X_test.shape)

3
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Train samples: (10000, 25088)
Test samples: (10000, 25088)


In [182]:
X_train = torch.from_numpy(X_train).to(device)
y_train = torch.from_numpy(y_train).to(device)
X_test = torch.from_numpy(X_test).to(device)
# y_test = torch.from_numpy(y_test).to(device)

In [183]:
W_init = torch.randn(X_train.shape[1], y_train.shape[1]).to(device)

### 10

In [184]:
T = 10
start = time.time()
model = SoftmaxTemperature()
model.fit(X_train, y_train, W_init, T, lr=10)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 10 loss: 0.4629456831361513
Epoch: 20 loss: 0.28194172088815234
Epoch: 30 loss: 0.19763418731832097
Epoch: 40 loss: 0.14855415523971396
Epoch: 50 loss: 0.11848412332186548
Epoch: 60 loss: 0.09752409943680872
Epoch: 70 loss: 0.08273375406970807
Epoch: 80 loss: 0.07157640395766361
Epoch: 90 loss: 0.0631306875991164
Epoch: 100 loss: 0.05633579167621712
Epoch: 110 loss: 0.0508610283329936
Epoch: 120 loss: 0.04629078313027444
Epoch: 130 loss: 0.042475288066896766
Epoch: 140 loss: 0.03922345028140166
Epoch: 150 loss: 0.03648972068700325
Epoch: 160 loss: 0.034043527180761986
Epoch: 170 loss: 0.031899884445771305
Epoch: 180 loss: 0.030013825071119857
Epoch: 190 loss: 0.0283522396921337
Epoch: 200 loss: 0.026848888646875662
Train completed in 20.491928815841675s


In [185]:
y_pred = model.predict(X_test)
accuracy_score(y_pred.cpu().numpy(), y_test)*100

65.4

### 1000

In [186]:
T = 1000
start = time.time()
model = SoftmaxTemperature()
model.fit(X_train, y_train, W_init, T, lr=10)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 10 loss: 1.9517201998651028
Epoch: 20 loss: 1.7400307272285225
Epoch: 30 loss: 1.5976185853391887
Epoch: 40 loss: 1.4937743986874819
Epoch: 50 loss: 1.4135168175362052
Epoch: 60 loss: 1.348747927978635
Epoch: 70 loss: 1.2947797964381054
Epoch: 80 loss: 1.2487224006732927
Epoch: 90 loss: 1.2086494816121647
Epoch: 100 loss: 1.1732457197662443
Epoch: 110 loss: 1.1415800226944033
Epoch: 120 loss: 1.1129547871784773
Epoch: 130 loss: 1.0868540585746524
Epoch: 140 loss: 1.0628761346898041
Epoch: 150 loss: 1.0407098206777126
Epoch: 160 loss: 1.0200951419275721
Epoch: 170 loss: 1.0008405143442798
Epoch: 180 loss: 0.9827709189273883
Epoch: 190 loss: 0.9657504744451261
Epoch: 200 loss: 0.9496619616830256
Train completed in 19.441881895065308s


In [187]:
y_pred = model.predict(X_test)
accuracy_score(y_pred.cpu().numpy(), y_test)*100

63.12

### 0.5

In [188]:
T = 0.5
start = time.time()
model = SoftmaxTemperature()
model.fit(X_train, y_train, W_init, T, lr=10)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 10 loss: 0.012035676618602885
Epoch: 20 loss: 0.006573236732344042
Epoch: 30 loss: 0.004755730650836655
Epoch: 40 loss: 0.003744843428055842
Epoch: 50 loss: 0.0031288090390412236
Epoch: 60 loss: 0.002694060940882784
Epoch: 70 loss: 0.002368920175278008
Epoch: 80 loss: 0.002120985014176918
Epoch: 90 loss: 0.0019141341739386392
Epoch: 100 loss: 0.0017496288263010855
Epoch: 110 loss: 0.0016134574826566265
Epoch: 120 loss: 0.0014957781106888575
Epoch: 130 loss: 0.001394853459762294
Epoch: 140 loss: 0.0013073195566880942
Epoch: 150 loss: 0.001231334702135746
Epoch: 160 loss: 0.0011633222296852821
Epoch: 170 loss: 0.001102608304913551
Epoch: 180 loss: 0.001049136986433895
Epoch: 190 loss: 0.0009997582458598417
Epoch: 200 loss: 0.0009548020293997681
Train completed in 18.15932869911194s


In [189]:
y_pred = model.predict(X_test)
accuracy_score(y_pred.cpu().numpy(), y_test)*100

59.29